# Sprint 論文読解入門

## 1.このSprintについて

### Sprintの目的
- 機械学習分野の論文から有益な情報を引き出せるようにする
- これまで扱ってきた領域の論文から新たな知識を得る

### どのように学ぶか
- ある論文に対しての問題に答えていくことで、読むポイントを学んでいきます。

## 2.論文読解

以下の論文を読み問題に答えてください。CNNを使った物体検出（Object Detection）の代表的な研究です。


[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99

<a href="https://arxiv.org/pdf/1506.01497.pdf">https://arxiv.org/pdf/1506.01497.pdf</a>

## 問題
それぞれについてJupyter Notebookにマークダウン形式で記述してください。

## 条件
- 答える際は論文のどの部分からそれが分かるかを書く。
- 必要に応じて先行研究（引用されている論文）も探しにいく。最低2つは他の論文を利用して回答すること。
- 論文の紹介記事を見ても良い。ただし、答えは論文内に根拠を探すこと。

### (1) 物体検出の分野にはどういった手法が存在したか。

- R-CNN
  - 特徴
    - 物体検出においてもCNNを活用できないか、という課題の先駆けとなった手法。主な処理は下記。
      1. Extract region proposal : まず、画像全体から物体っぽい領域を検出(selective search)
      2. Classify regions : CNNにより特徴量を抽出し、各領域内の物体カテゴリを認識
      3. Compute bounding box regression : 物体の位置、大きさを正確に求めるために1.で求めたBounding boxの座標を回帰により再計算
  - 課題
    - トレーニングには非常に多くのスペースと時間が必要。
    - オブジェクトの検出が遅い。

  
  
- SPP-net
  - "spatial pyramid pooling"を最後のCNN層の前に設置し、その後の全結合層へ渡すデータを固定サイズとすることで、入力画像を任意サイズとすることに成功
  - 計算を共有することによりR-CNNを高速化するために提案された。


- Fast R-CNN
  - R-CNN、SPPnetよりも高い検出品質（mAP）
  - トレーニングは、multi-task loss を使用し、シングルステージで行う
  - トレーニングはすべてのネットワーク層を更新できる
  - 特徴量のキャッシュにディスクストレージは不要


> State-of-the-art object detection networks depend on region proposal algorithms to hypothesize object locations.
Advances like SPPnet [1] and Fast R-CNN [2] have reduced the running time of these detection networks, exposing region
proposal computation as a bottleneck. 



### (2) Fasterとあるが、どういった仕組みで高速化したのか。

これまでの手法では領域提案の計算が最大のボトルネックであったため、Selective Searchなどの領域提案手法をRPN(Region Proposal Network)という小さな畳み込みネットワークに入れ替えて、RPN上で領域提案と物体検出の両方を行うようにした。





### (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

【解答】
- One-Stage=OverFeat, Two-Stage=RPNとして論じられている。
- この二つのregion proposalの予測に関する違いは下記。
  - OverFeatでは1つのアスペクト比の畳み込みフィルタのみから取得される。
  - RPNでは1つのアスペクト比の畳み込みフィルタと、異なるアスペクト比・縮尺のAnchorから取得される。


【補足】
- OverFeat
  - OverFeatの論文より
    - 分類、ローカリゼーション、検出に畳み込みネットワークを使用するための統合フレームワーク。（マルチスケールのスライディングウィンドウを使用）
    - オブジェクトの境界を予測することを学ぶことによるローカリゼーションへの新しい深層学習アプローチ。 
    - 単一の共有ネットワークを使用して、さまざまなタスクを同時に学習できることを示します。 この統合フレームワークが勝者です
  - Faster R-CNNの論文より
    - 畳み込みフィルタのスライディングウィンドウで回帰と分類子を使用する検出方法
    - 領域ごとの特徴は、スケールピラミッド上の1つのアスペクト比のスライディングウィンドウから取得される。


- RPN
  - 正方形(3×3)のスライディングウィンドウと、異なる縮尺とアスペクト比のAnchorによって領域提案を予測する。

### (4) RPNとは何か。
【解答】

- Two-Stageによる精度の高い領域提案を生成し、fast R-CNNへデータを渡す役割を持つCNN構造を用いたレイヤー。
- アーキテクチャは、n×nたたみ込み層と、それに続く2つの兄弟1×1たたみ込み層（それぞれ、regおよびcls）で実装されます。
- スライディングウィンドウによる特徴マップで抽出した特徴は、2つの兄弟の完全に接続されたレイヤー（ボックス回帰レイヤー（reg）とボックス分類レイヤー（cls））に供給される。
- RPNはFast R-CNNのネットワークと統合するために、「領域提案のファインチューニング」と「オブジェクト検出のファインチューニング」を交互に行う。
- つまり「objectが画像のどこにあるか」をRPNが検出・学習し、そのobjectが何なのか？についてはfast R-CNN側で分類・学習する。

### (5) RoIプーリングとは何か。
Roi pooling layerは、Maxプーリングを使用して、有効なROI内の特徴量をH×Wの固定空間範囲の小さな特徴マップに変換する。

※つまり、SPP-netで言う“spatial pyramid pooling”の替わり?

> The RoI pooling layer uses max pooling to convert the
features inside any valid region of interest into a small feature map with a fixed spatial extent of H × W (e.g., 7 × 7),
where H and W are layer hyper-parameters that are independent of any particular RoI. 

### (6) Anchorのサイズはどうするのが適切か。

- 特徴マップのH×W、各スライドのanchor数（=$k$）の乗算（$=H*W*k$）が最適なanchorサイズ
- 各スライドのanchor(k)は、デフォルトではk=9（3種類の縦横比、3種類の縮尺）
- ※デフォルトのボックス領域は1282、2562、および5122ピクセルの3つのスケールと、1：1、1：2、および2：1の3つのアスペクト比を使用している。

> The k proposals are parameterized relative to k reference boxes, which we call anchors. An anchor is centered at the sliding window
in question, and is associated with a scale and aspect
ratio (Figure 3, left). By default we use 3 scales and
3 aspect ratios, yielding k = 9 anchors at each sliding
position. For a convolutional feature map of a size
W × H (typically ∼2,400), there are WHk anchors in
total.

### (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

- データセット
  - MS COCO
  

- 先行研究と比べた指標値

<img src="./img/result.png"></img>

### (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

※今回は割愛させていただきます。追々調べます。

## 以降は学習メモ

## ※訳・意味（メモ）
- object proposal
  - 物体候補、物体提案、物体候補提案
  - Selective Search, Exhaustive Searchなどが、object proposal アルゴリズム


- region proposal
  - 領域候補、領域提案、領域候補提案
  - 画像の中のたくさんのボックス(bounding box)の候補をリストアップすること
  
  
- End-to-End
  > End-to-End学習とは端つまり入力と出力だけ渡して、途中で発生する処理全てを学習してしまうというもの。
    OCRを例に挙げると、入力の画像から途中の処理を細かいタスクに分け最終的に文字認識に到るという構造が
    一般的、しかしEnd-to-End学習では中間の処理も全て学習させる。
    注意点としてデータセットがたくさん必要になる点、逆に言うとデータセットが少ないなら今までの手法を使えばよい。
   
<a href="https://qiita.com/wakame1367/items/5a3844e1f56336290dfb"><img src="./img/endtoend.png"></a>


## ※補足情報
- 全般
  - <a href="https://qiita.com/mshinoda88/items/9770ee671ea27f2c81a9">物体検出についての歴史まとめ(1)</a>
  
  
- Sliding window method
  - Exhaustive Search
    - objectは画像内の任意の位置と縮尺で配置できるため(つまり、objectが画像のどこにあるかわからないから)、全体を**「しらみつぶしに探索」**するのは当然。ただし、探索スペースが巨大で計算コストが非常に高い。<br>※ selective searchの論文より
  <a href="http://www.huppelen.nl/publications/selectiveSearchDraft.pdf">http://www.huppelen.nl/publications/selectiveSearchDraft.pdf</a>
  > As an object can be located at any position and scale in the image,
it is natural to search everywhere [8, 16, 36]. However, the visual
search space is huge, making an exhaustive search computationally
expensive. This imposes constraints on the evaluation cost per location and/or the number of locations considered. Hence most of
these sliding window techniques use a coarse search grid and fixed
aspect ratios, using weak classifiers and economic image features such as HOG [8, 16, 36].



- HOG
  - HOG（Histogram of Oriented Gradients）特徴は、輝度の勾配方向の分布により特徴を捉えます。<br><a href="https://www.pro-s.co.jp/blog/system/opencv/6338">https://www.pro-s.co.jp/blog/system/opencv/6338</a>



- Selective Search method
  - ピクセルレベルで類似する領域をグルーピングして候補領域を選出
  - <a href="http://www.huppelen.nl/publications/selectiveSearchDraft.pdf">http://www.huppelen.nl/publications/selectiveSearchDraft.pdf</a>
  - ざっくり下記ステップで物体領域候補を提案する。<br><a href="https://qiita.com/FukuharaYohei/items/37c32cda989de614d497">https://qiita.com/FukuharaYohei/items/37c32cda989de614d497</a>
    1. 画像を小さい領域郡に分割
    2. 隣り合う領域間の類似度を計算
    3. 最も類似度の高い領域を統合
    4. 2と3を領域が1つになるまで繰り返す
    <br><img src="./img/selective_search.png" width="1000" height="500"></img>

## R-CNN

<a href="https://arxiv.org/abs/1311.2524">論文：https://arxiv.org/abs/1311.2524</a>

<img src="./img/rcnn.png"></img>

1. 入力画像を取得
2. 約2000のボトムアップ？のregion proposalsを抽出<br>2.5 領域画像を全て同じサイズ(正方形)にリサイズ
3. 大規模のCNNを使用して各region proposalsの特徴量を計算する
4. 各領域のクラス分類をする（クラス固有？の線形SVMを使用して）

> Figure 1: Object detection system overview. Our system (1)
takes an input image, (2) extracts around 2000 bottom-up region
proposals, (3) computes features for each proposal using a large
convolutional neural network (CNN), and then (4) classifies each
region using class-specific linear SVMs. R-CNN achieves a mean
average precision (mAP) of 53.7% on PASCAL VOC 2010. For
comparison, [39] reports 35.1% mAP using the same region proposals, but with a spatial pyramid and bag-of-visual-words approach. The popular deformable part models perform at 33.4%.
On the 200-class ILSVRC2013 detection dataset, R-CNN’s
mAP is 31.4%, a large improvement over OverFeat [34], which
had the previous best result at 24.3%.


【問題点】※Fast R-CNNの論文より
- トレーニングには非常に多くのスペースと時間が必要。SVMとbounding-box regressor(独立変数？)のトレーニングのために、各画像中の各object proposalから特徴が抽出され、ディスクに書き込まれる。このプロセスには、VGG16のような非常に深いネットワークでは、5k images of the VOC07 trainval set あたり、2.5GPUかかる。これらには数百GBのストレージが必要。
- オブジェクトの検出が遅い。VGG16での検出には47秒/image（GPU上）が必要。R-CNNはConvNet転送を実行するため低速。計算を共有せずに、各region proposalsに渡します。
- 空間ピラミッドプーリングネットワーク（SPPnets）は、計算を共有することによりR-CNNを高速化するために提案された。

> The Region-based Convolutional Network method (RCNN) [9] achieves excellent object detection accuracy by
using a deep ConvNet to classify object proposals. R-CNN,
however, has notable drawbacks:
> 1. Training is a multi-stage pipeline. R-CNN first finetunes a ConvNet on object proposals using log loss.
Then, it fits SVMs to ConvNet features. These SVMs
act as object detectors, replacing the softmax classifier learnt by fine-tuning. In the third training stage,
bounding-box regressors are learned.
> 2. Training is expensive in space and time. For SVM
and bounding-box regressor training, features are extracted from each object proposal in each image and
written to disk. With very deep networks, such as
VGG16, this process takes 2.5 GPU-days for the 5k
images of the VOC07 trainval set. These features require hundreds of gigabytes of storage.
> 3. Object detection is slow. At test-time, features are
extracted from each object proposal in each test image.
Detection with VGG16 takes 47s / image (on a GPU).
R-CNN is slow because it performs a ConvNet forward
pass for each object proposal, without sharing computation.
Spatial pyramid pooling networks (SPPnets) [11] were proposed to speed up R-CNN by sharing computation. 

## SPP-net
<a href="https://arxiv.org/pdf/1406.4729.pdf">https://arxiv.org/pdf/1406.4729.pdf</a>

- 既存のCNNでは入力画像サイズが固定である必要があった。この要件によって、画像や任意のサイズのサブ画像への認識精度が低下する可能性がある。
- SSP-netはネットワークに別のpooling strategy "spatial pyramid pooling" を備えており、任意の画像サイズを入力にしても認識精度を低下させずに学習できる技術。
- R-CNNメソッドより高速（24〜102倍）でありながら、それ以上・あるいは同等の精度を持つ。

> Existing deep convolutional neural networks (CNNs) require a fixed-size (e.g., 224×224) input image. This requirement is “artificial” and may reduce the recognition accuracy for the images or sub-images of an arbitrary size/scale. In this
work, we equip the networks with another pooling strategy, “spatial pyramid pooling”, to eliminate the above requirement. The
new network structure, called SPP-net, can generate a fixed-length representation regardless of image size/scale.

>  In processing test images, our method is 24-102x faster than the R-CNN method, while achieving better or comparable accuracy on Pascal VOC 2007. 


下記をざっくり要約すると「CNNへの入力画像のサイズが固定である必要があるのは、そのあとの全結合層(SVMも)への入力サイズを指定する必要があるからなので、最後のCNN層の前に出力が固定長になってくれるような層 "spatial pyramid pooling"(SPP)を入れれば解決するじゃん？」となる。

>  So why do CNNs require a fixed input size? A CNN mainly consists of two parts: convolutional layers, and fully-connected layers that follow.The convolutional layers operate in a sliding-window manner and output feature maps which represent the spatial arrangement of the activations (Figure 2). In fact, convolutional layers do not require a fixed image size and can generate feature maps of any sizes.On the other hand, the fully-connected layers need to have fixed-size/length input by their definition. Hence, the fixed-size constraint comes only from the fully-connected layers, which exist at a deeper stage of the network.
<br>
>  In this paper, we introduce a spatial pyramid pooling (SPP) [14, 15] layer to remove the fixed-size constraint of the network.  Specifically, we add an SPP layer on top of the last convolutional layer.  The SPP layer pools the features and generates fixed-length outputs, which are then fed into the fully-connected layers (or other classifiers).

可変サイズのイメージでトレーニングすると、スケールの不変性が高まり、過剰適合が減少します。

> Training with variable-size images increases scale-invariance and reduces over-fitting.

※メモ
- なぜ高速化？プーリング以外にも処理があるの？計算を共有するとは？？

【欠点】

- R-CNNと同様に、トレーニングは、特徴量の抽出、log lossによるネットワークのファインチューニング、SVMのトレーニング、最後にbounding-box 回帰のフィッティングを含むマルチステージパイプライン。
- 機能もディスクに書き込まれる。
- しかし、R-CNNとは異なり、SPP-netにおけるファインチューニングのアルゴリズムは、spatial pyramid poolingに先行する畳み込み層を更新できない。
- 当然のことながら、この制限（固定された畳み込み層）は非常に深いネットワークの精度を制限する。

> SPPnet also has notable drawbacks. Like R-CNN, training is a multi-stage pipeline that involves extracting features, fine-tuning a network with log loss, training SVMs, and finally fitting bounding-box regressors. Features are also written to disk. But unlike R-CNN, the fine-tuning algorithm proposed in cannot update the convolutional layers that precede the spatial pyramid pooling.  Unsurprisingly, this limitation (fixed convolutional layers) limits the accuracy of very deep networks.

## Fast R-CNN

<a href="https://arxiv.org/pdf/1504.08083.pdf">https://arxiv.org/pdf/1504.08083.pdf</a>


【概要】

> This paper proposes a Fast Region-based Convolutional
Network method (Fast R-CNN) for object detection. Fast
R-CNN builds on previous work to efficiently classify object proposals using deep convolutional networks. Compared to previous work, Fast R-CNN employs several innovations to improve training and testing speed while also
increasing detection accuracy. Fast R-CNN trains the very
deep VGG16 network 9× faster than R-CNN, is 213× faster
at test-time, and achieves a higher mAP on PASCAL VOC
2012. Compared to SPPnet, Fast R-CNN trains VGG16 3×
faster, tests 10× faster, and is more accurate. Fast R-CNN
is implemented in Python and C++ (using Caffe) and is
available under the open-source MIT License at https:
//github.com/rbgirshick/fast-rcnn.

【特徴】
1. R-CNN、SPPnetよりも高い検出品質（mAP）
2. トレーニングは、multi-task loss を使用し、シングルステージで行う
3. トレーニングはすべてのネットワーク層を更新できる
4. 特徴量のキャッシュにディスクストレージは不要

2により、classification、bounding box regressionを同時に学習することができるようになったが、region proposalには既存のselective searchを利用しているため、
画像像の入力から物体候補抽出までと、物体候補から物体の識別までが分断されてしまっており、画像の入力から物体検出までを1つのモデルで一気に学習・推定することができるEnd-to-Endには至っていなかった。

> 1. Higher detection quality (mAP) than R-CNN, SPPnet 
> 2. Training is single-stage, using a multi-task loss
> 3. Training can update all network layers
> 4. No disk storage is required for feature caching

【THE ROI POOLING LAYER】

Roi pooling layerは、Maxプーリングを使用して、（inside any valid region of interest？？の）特徴量をH×Wの固定空間範囲の小さな特徴マップに変換します。

※つまり、SPP-netで言う“spatial pyramid pooling”の替わり。

> The RoI pooling layer uses max pooling to convert the features inside any valid region of interest into a small feature map with a fixed spatial extent of   H  ×  W   (\eg,   7  ×  7  ), where   H   and   W   are layer hyper-parameters that are independent of any particular RoI. In this paper, an RoI is a rectangular window into a conv feature map. Each RoI is defined by a four-tuple   (  r  ,  c  ,  h  ,  w  )   that specifies its top-left corner   (  r  ,  c  )   and its height and width   (  h  ,  w  )  .    RoI max pooling works by dividing the   h  ×  w   RoI window into an   H  ×  W   grid of sub-windows of approximate size   h  /  H  ×  w  /  W   and then max-pooling the values in each sub-window into the corresponding output grid cell. Pooling is applied independently to each feature map channel, as in standard max pooling. The RoI layer is simply the special-case of the spatial pyramid pooling layer used in SPPnets [11] in which there is only one pyramid level. We use the pooling sub-window calculation given in [11].  


## faster R-CNNとは

- RPNとFast R-CNNを統合した物体検出ネットワーク
- 統合するためにregion proposalタスクと、オブジェクト検出のファインチューニングを交互に行うトレーニングスキームとしている。
- 両タスク間で共有される畳み込みの特徴量を持つ統合ネットワークを持つ。


> To unify RPNs with Fast R-CNN [2] object detection networks, we propose a training scheme that alternates between fine-tuning for the region proposal task and then fine-tuning for object detection, while keeping the proposals fixed. This scheme converges quickly and produces a unified network with convolutional features that are shared between both tasks.


- Faster R-CNNの物体検出システムは2つのモジュールで構成される。 
- 最初のモジュールはregion proposalをする深い完全畳み込みネットワーク
- 2番目のモジュールは、fast R-CNN
- システム全体は、オブジェクト検出のための単一の統合ネットワーク（図2）。
- 最近人気のある神経の用語を使用して‘attention’メカニズムを備えたネットワーク、RPN moduleは、Fast R-CNNモジュールに探す場所を伝えます。
- セクション3.1では、デザインとプロパティを紹介します
地域提案のためのネットワークの。 セクション3.2で
両方のモジュールをトレーニングするためのアルゴリズムを開発する
機能を共有しました。

> Our object detection system, called Faster R-CNN, is
composed of two modules. The first module is a deep
fully convolutional network that proposes regions,
and the second module is the Fast R-CNN detector [2]
that uses the proposed regions. The entire system is a single, unified network for object detection (Figure 2).
Using the recently popular terminology of neural
networks with ‘attention’ [31] mechanisms, the RPN
module tells the Fast R-CNN module where to look.
In Section 3.1 we introduce the designs and properties
of the network for region proposal. In Section 3.2 we
develop algorithms for training both modules with
features shared.


## RPNとは



【前提】
- 最先端の（現状の）物体検出ネットワークはregion proposalアルゴリズムに依存してオブジェクトの場所を仮定している。
- SPPnetやFast R-CNNなどの進歩により、これらの検出ネットワークの実行時間が短縮されたものの、resion proposalの計算がボトルネックになっている。
- この作業(今回の論文？)では、検出ネットワークとフルイメージ convolutional featuresを共有するregion proposal network（RPN）を導入し、ほぼ無料のregion proposalを可能にします。
- RPNは完全畳み込みネットワークで、各位置でオブジェクトの境界とオブジェクトネススコアを同時に予測します。
- RPNはend-to-endでトレーニングされ、faster R-CNNが検出に使用する高品質のregion proposalを生成します。（つまり、RPNがregion proposalを生成して次の処理に渡す）

> State-of-the-art object detection networks depend on region proposal algorithms to hypothesize object locations.<br>
> Advances like SPPnet [1] and Fast R-CNN [2] have reduced the running time of these detection networks, exposing region
proposal computation as a bottleneck. <br>
> In this work, we introduce a Region Proposal Network (RPN) that shares full-image
convolutional features with the detection network, thus enabling nearly cost-free region proposals. <br>
> An RPN is a fully convolutional network that simultaneously predicts object bounds and objectness scores at each position. <br>
> The RPN is trained end-to-end to enerate high-quality region proposals, which are used by Fast R-CNN for detection.<br>

- RPNは様々なスケール・アスペクト比でも効率的に予測できるよう設計されている。RPNは従来の一般的な方法（画像ピラミッド・フィルタピラミッド）とは異なり、“anchor” boxes を使用している。

> RPNs are designed to efficiently predict region proposals with a wide range of scales and aspect ratios. In
contrast to prevalent methods [8], [9], [1], [2] that use pyramids of images (Figure 1, a) or pyramids of filters
(Figure 1, b), we introduce novel “anchor” boxes
that serve as references at multiple scales and aspect
ratios. 

- RPNはFast R-CNNのネットワークと統合するために、「領域提案のファインチューニング」と「オブジェクト検出のファインチューニング」を交互に行う。

> To unify RPNs with Fast R-CNN [2] object detection networks, we propose a training scheme that
alternates between fine-tuning for the region proposal
task and then fine-tuning for object detection, while
keeping the proposals fixed. This scheme converges
quickly and produces a unified network with convolutional features that are shared between both tasks.1

- 物体検出の精度を向上することができる、ディープラーニングをベースとした物体検出システム。畳み込み機能を下層の検出ネットワークと共有することでほぼ無料で利用でき、またほぼリアルタイムで実行できる。

> We have presented RPNs for efficient and accurate
region proposal generation. By sharing convolutional features with the down-stream detection network, the
region proposal step is nearly cost-free. Our method
enables a unified, deep-learning-based object detection system to run at near real-time frame rates. The
learned RPN also improves region proposal quality
and thus the overall object detection accuracy.


入力画像を畳み込みニューラルネットワークに通して、出力したfeaturemapを複数の縦横比と縮尺で範囲抽出したアンカーを複数作成してRoi Pooling層に引き渡す。

- RPNは、（任意のサイズの）画像を入力として受け取り、それぞれが客観性スコアを持つ長方形のobject proposalのセットを出力します。
- 多くの方法で一般的であるように、長方形の領域を検討してください（たとえば、[27、4、6]）。
- 「オブジェクト性」は、一連のオブジェクトクラスとバックグラウンドのメンバーシップを測定します。
- このプロセスは、このセクションで説明する完全たたみ込みネットワーク[7]を使用してモデル化します。
- 私たちの最終的な目標は、fas R-CNNオブジェクト検出ネットワーク[2]と計算を共有することなので、両方のネットが共通の畳み込み層のセットを共有すると想定します。
- 私たちの実験では、5つの共有可能な畳み込み層をもつZeiler and Fergusモデル[32]（ZF）と、13の共有可能な畳み込み層をもつSimonyan and Zissermanモデル[3]（VGG-16）を調べます。
- 地域提案を生成するために、最後の共有畳み込み層によって出力された畳み込み特徴マップ上に小さなネットワークをスライドさせます。
- この小さなネットワークは、入力として、入力たたみ込みフィーチャマップのn×n空間ウィンドウを受け取ります。各スライディングウィンドウは、低次元の機能にマッピングされます（ZFの場合は256-d、VGGの場合は512-d、ReLU [33]を使用）
- この機能は、2つの兄弟の完全に接続されたレイヤー（ボックス回帰レイヤー（reg）とボックス分類レイヤー（cls））に供給されます。このペーパーでは、n = 3を使用していますが、入力画像の有効受容野は大きい（ZFおよびVGGの場合はそれぞれ171および228ピクセル）。このミニネットワークは、図3（左）の1つの位置に示されています。ミニネットワークはスライディングウィンドウ方式で動作するため、完全に接続されたレイヤーはすべての空間位置で共有されます。このアーキテクチャは、n×nたたみ込み層と、それに続く2つの兄弟1×1たたみ込み層（それぞれ、regおよびcls）で実装されます。

> REGION PROPOSAL NETWORKS  A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score.3  3“Region” is a generic term and in this paper we only consider rectangular regions, as is common for many methods (e.g., [27, 4, 6]). “Objectness” measures membership to a set of object classes vs. background. We model this process with a fully convolutional network [7], which we describe in this section. Because our ultimate goal is to share computation with a Fast R-CNN object detection network [2], we assume that both nets share a common set of convolutional layers. In our experiments, we investigate the Zeiler and Fergus model [32] (ZF), which has 5 shareable convolutional layers and the Simonyan and Zisserman model [3] (VGG-16), which has 13 shareable convolutional layers.  To generate region proposals, we slide a small network over the convolutional feature map output by the last shared convolutional layer. This small network takes as input an   n  ×  n spatial window of the input convolutional feature map. Each sliding window is mapped to a lower-dimensional feature (256-d for ZF and 512-d for VGG, with ReLU [33] following). This feature is fed into two sibling fully-connected layers—a box-regression layer (reg) and a box-classification layer (cls). We use   n  =  3 in this paper, noting that the effective receptive field on the input image is large (171 and 228 pixels for ZF and VGG, respectively). This mini-network is illustrated at a single position in Figure 3 (left). Note that because the mini-network operates in a sliding-window fashion, the fully-connected layers are shared across all spatial locations. This architecture is naturally implemented with an   n  ×  n convolutional layer followed by two sibling   1  ×  1 convolutional layers (for reg and cls, respectively).

## Anchors
各スライディングウィンドウの場所で、同時に複数の地域提案を予測する
各場所の可能な最大提案の
kとして示されます。 したがって、regレイヤーには4k出力エンコーディングがあります
kボックスの座標、およびclsレイヤー出力
オブジェクトの確率を推定する2kスコア
各提案のオブジェクト4
。 k個の提案は、アンカーと呼ばれるk個の参照ボックスに対してパラメーター化されます。 アンカーはスライディングウィンドウの中央に配置されます
問題であり、スケールとアスペクトに関連付けられています
比率（図3、左）。 デフォルトでは3つのスケールを使用し、
3つのアスペクト比、各スライドでk = 9アンカー
ポジション。 サイズの畳み込み特徴マップの場合
W×H（通常〜2,400）、W Hkアンカーが
合計。

> Anchors
At each sliding-window location, we simultaneously
predict multiple region proposals, where the number
of maximum possible proposals for each location is
denoted as k. So the reg layer has 4k outputs encoding
the coordinates of k boxes, and the cls layer outputs
2k scores that estimate probability of object or not
object for each proposal4
. The k proposals are parameterized relative to k reference boxes, which we call anchors. An anchor is centered at the sliding window
in question, and is associated with a scale and aspect
ratio (Figure 3, left). By default we use 3 scales and
3 aspect ratios, yielding k = 9 anchors at each sliding
position. For a convolutional feature map of a size
W × H (typically ∼2,400), there are W Hk anchors in
total.


> For anchors, we use 3 scales with box areas of 1282
,
2562
, and 5122 pixels, and 3 aspect ratios of 1:1, 1:2,
and 2:1. These hyper-parameters are not carefully chosen for a particular dataset, and we provide ablation
experiments on their effects in the next section. As discussed, our solution does not need an image pyramid
or filter pyramid to predict regions of multiple scales,
saving considerable running time. Figure 3 (right)
shows the capability of our method for a wide range
of scales and aspect ratios. Table 1 shows the learned
average proposal size for each anchor using the ZF
net. We note that our algorithm allows predictions
that are larger than the underlying receptive field.
Such predictions are not impossible—one may still
roughly infer the extent of an object if only the middle
of the object is visible.

## One-Stage Detection vs. Two-Stage Proposal + Detection. 

<p>OverFeat 論文
<a href="https://arxiv.org/pdf/1312.6229.pdf">https://arxiv.org/pdf/1312.6229.pdf</a>
</p>

分類、ローカリゼーション、検出にたたみ込みネットワークを使用するための統合フレームワークを提示します。 マルチスケールのスライディングウィンドウ
アプローチは、ConvNet内で効率的に実装できます。 また、
オブジェクトの境界を予測することを学ぶことによるローカリゼーションへの新しい深層学習アプローチ。 境界ボックスは抑制されるのではなく蓄積され、
検出の信頼性を高めます。 単一の共有ネットワークを使用して、さまざまなタスクを同時に学習できることを示します。 この統合フレームワークが勝者です
大規模視覚認識チャレンジのローカリゼーションタスクの概要
2013（ILSVRC2013）を検出し、検出と
分類タスク。 競技後の仕事で、私たちは新しい最先端の技術を確立します
検出タスク用。 最後に、最高のモデルから特徴抽出器をリリースします
OverFeatと呼ばれます。

> We present an integrated framework for using Convolutional Networks for classification, localization and detection. We show how a multiscale and sliding window
approach can be efficiently implemented within a ConvNet. We also introduce a
novel deep learning approach to localization by learning to predict object boundaries. Bounding boxes are then accumulated rather than suppressed in order to
increase detection confidence. We show that different tasks can be learned simultaneously using a single shared network. This integrated framework is the winner
of the localization task of the ImageNet Large Scale Visual Recognition Challenge
2013 (ILSVRC2013) and obtained very competitive results for the detection and
classifications tasks. In post-competition work, we establish a new state of the art
for the detection task. Finally, we release a feature extractor from our best model
called OverFeat.




> One-Stage Detection vs. Two-Stage Proposal + Detection. <br>
>The OverFeat paper [9] proposes a detection method that uses regressors and classifiers on sliding windows over convolutional feature maps. OverFeat is a one-stage, class-specific detection pipeline, and ours is a two-stage cascade consisting of class-agnostic proposals and class-specific detections. In OverFeat, the region-wise features come from a sliding window of one aspect ratio over a scale pyramid. These features are used to simultaneously determine the location and category of objects. In RPN, the features are from square (3
×
>3) sliding windows and predict proposals relative to anchors with different scales and aspect ratios. Though both methods use sliding windows, the region proposal task is only the first stage of Faster R-CNN—the downstream Fast R-CNN detector attends to the proposals to refine them. In the second stage of our cascade, the region-wise features are adaptively pooled [1, 2] from proposal boxes that more faithfully cover the features of the regions. We believe these features lead to more accurate detections.
>To compare the one-stage and two-stage systems, we emulate the OverFeat system (and thus also circumvent other differences of implementation details) by one-stage Fast R-CNN. In this system, the “proposals” are dense sliding windows of 3 scales (128, 256, 512) and 3 aspect ratios (1:1, 1:2, 2:1). Fast R-CNN is trained to predict class-specific scores and regress box locations from these sliding windows. Because the OverFeat system adopts an image pyramid, we also evaluate using convolutional features extracted from 5 scales. We use those 5 scales as in [1, 2].
>Table X compares the two-stage system and two variants of the one-stage system. Using the ZF model, the one-stage system has an mAP of 53.9%. This is lower than the two-stage system (58.7%) by 4.8%. This experiment justifies the effectiveness of cascaded region proposals and object detection. Similar observations are reported in [2, 39], where replacing SS region proposals with sliding windows leads to 
∼
6% degradation in both papers. We also note that the one-stage system is slower as it has considerably more proposals to process.